In [5]:
from utilities_IGNOBEL import *
with open('credential.json','r') as f:
    cred = json.load(f)
cluster = MongoClient(cred['cred'])
db = cluster['Game']
collection = db['Players']

In [4]:
"""Usare queste rose se FG non ha quelle corrette, e.g. durante il mercato"""
#RoseSS = rose_complete()

#RoseFG = rose()

#labels = RoseFG.columns
#Rose = RoseSS[labels]

'Usare queste rose se FG non ha quelle corrette, e.g. durante il mercato'

In [4]:
#infortunati(giornata=1) #restituisce una singola lista con tutti gli infortunati di ogni squadra di serie A per la giornata indicata

In [2]:
#rose() #restituisce le rose attuali di ogni fantallenatore

In [3]:
#formazioni() #restituisce le formazioni schierate in ogni giornata (inclusa la panchina)

In [12]:
#per qualche strano motivo a volte da errore la prima volta che gira, 
#ma funziona le volte successive, potrebbe essere un caso
V = bonus_panchina(giornata=21) 
V
# restituisce i voti dei panchinari che non sono scesi in campo (link diverso temp)

,IGNORANZA EVERYWHERE,MAINZ NA GIOIA,XYZ,AS 800A,PALLA PAZZA,I DISEREDATI,SOROS FC,PDG 1908
Bonus Panchinari,0.5,0,0,1.5,0,1.5,0,0


In [35]:
G = goal_subiti(giornata=21)
G
# restituisce il totale dei gol subiti da ogni portiere sceso in campo

,IGNORANZA EVERYWHERE,MAINZ NA GIOIA,XYZ,AS 800A,PALLA PAZZA,I DISEREDATI,SOROS FC,PDG 1908
Goal subiti,1,2,2,0,0,0,0,0


In [36]:
M = modificatore(giornata=21) 
M
# restituisce il bonus modificatore difesa guadagnato da ogni squadra

,IGNORANZA EVERYWHERE,MAINZ NA GIOIA,XYZ,AS 800A,PALLA PAZZA,I DISEREDATI,SOROS FC,PDG 1908
Modificatore,0,0.5,0.5,0,1.5,1.5,0,1.5


In [37]:
C = cartellini(giornata=21)
C

,IGNORANZA EVERYWHERE,MAINZ NA GIOIA,XYZ,AS 800A,PALLA PAZZA,I DISEREDATI,SOROS FC,PDG 1908
C. gialli,0,3,2,1,1,0,0,0
C. rossi,1,0,0,0,0,0,0,0


In [2]:
CI = count_inf(infortunati(giornata = 28), rose()) #restituisce il numero di infortunati in ogni rosa
##NOTA: usa Rose definito sopra nel caso in cui FG non ha le rose corrette e quindi serve usare lo spreadsheet
CI

,XYZ,MAINZ NA GIOIA,I DISEREDATI,PALLA PAZZA,AS 800A,PDG 1908,SOROS FC,IGNORANZA EVERYWHERE
tot Infortunati,0,0,0,0,0,0,0,0


In [10]:
infortunati(giornata = 28)

[]

In [39]:
F = fantapunti_fatti(giornata=21)

In [41]:
S = fantapunti_subiti(giornata=21)

In [42]:
#tot = IGNOBEL_tot(giornata=6) # a causa di ritardi di connessione, potrebbe dare errore le prime volte: in tal caso far girare nuovamente
output = pd.concat([F,S,G,C,V,M,CI], axis = 0).T #enormi problemi di connessione, quindi ho dovuto dividere in singole tasks
tot = output.T
tot

,IGNORANZA EVERYWHERE,MAINZ NA GIOIA,XYZ,AS 800A,PALLA PAZZA,I DISEREDATI,SOROS FC,PDG 1908
Fantapunti Fatti,64.0,69.0,64.00,71.25,72.75,75.50,67.50,72.75
Fantapunti Subiti,69.0,64.0,71.25,64.00,75.50,72.75,72.75,67.50
Goal subiti,1.0,2.0,2.00,0.00,0.00,0.00,0.00,0.00
C. gialli,0.0,3.0,2.00,1.00,1.00,0.00,0.00,0.00
C. rossi,1.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00
Bonus Panchinari,0.5,0.0,0.00,1.50,0.00,1.50,0.00,0.00
Modificatore,0.0,0.5,0.50,0.00,1.50,1.50,0.00,1.50
tot Infortunati,4.0,4.0,1.00,4.00,1.00,4.00,2.00,5.00


In [2]:
giornata=35
tot = IGNOBEL_tot(giornata)
tot

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[7]/main/div[3]/div[2]/div[1]/div[1]/div/div[2]/div[2]/div[1]/div[1]/div/div[2]/h4"}
  (Session info: headless chrome=92.0.4515.107)


In [5]:
tot.to_pickle("Dati_storici/Giornata_"+str(giornata)+".pkl")
#crea 8 dataframe individuali con i dati aggiornati fino alla giornata indicata
aggiorna_database(giornata)

Dati aggiornati fino alla 35 giornata


In [4]:
G = pd.read_pickle("Dati_storici/Giornata_22.pkl")
G

,PDG 1908,AS 800A,SOROS FC,IGNORANZA EVERYWHERE,PALLA PAZZA,MAINZ NA GIOIA,XYZ,I DISEREDATI
Fantapunti Fatti,72.00,72.25,65.75,71.75,68.50,71.75,75.75,73.25
Fantapunti Subiti,72.25,72.00,71.75,65.75,71.75,68.50,73.25,75.75
Goal subiti,2.00,1.00,3.00,0.00,0.00,1.00,0.00,1.00
C. gialli,1.00,1.00,0.00,1.00,0.00,1.00,1.00,1.00
C. rossi,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Bonus Panchinari,0.00,2.25,3.50,0.00,1.50,1.00,0.00,3.00
Modificatore,0.50,0.00,0.00,1.50,1.00,0.50,1.50,1.00
tot Infortunati,4.00,4.00,2.00,4.00,1.00,4.00,2.00,4.00


In [5]:
#voti_all = scarica_voti(giornata = 3, stagione = '2020-21')
#scarica il dataframe con tutti i voti direttamente da fantacalcio.it

In [6]:
storico_individuale(nome = 'luca', giornata = 22) 
#crea un dataframe individuale con le informazioni utili per gli IGNOBEL, con i dati fino alla giornata scelta
#i dati vengono pescati dai pkl contenuti nella cartella Dati_storici

,gg,pf,ps,gs,c,pan,mod,inf,nome
1,1,72.25,67.5,0,6,0,0,7,luca
2,2,77,64.25,2,1,0,0,5,luca
3,3,73.5,70.5,1,2,1.75,0,3,luca
4,4,65,65,0,2,3,0.5,4,luca
5,5,66,67.25,2,2,0,0,4,luca
6,6,70.25,71,1,2,1.5,0.5,5,luca
7,7,69,67.25,1,1,1.5,0,4,luca
8,8,69.75,73.75,1,2,0.5,0,3,luca
9,9,66.5,62.5,2,4,1,0,2,luca
10,10,67.25,70.75,1,1,0,1,2,luca


In [12]:
#include svincolati
#ST = stats_by_team_full(primavera = True)
ST_INFO = stats_by_team_NO_INFO(stagione = '2021-22',primavera = True) #NO PERSONAL DATA
#missing_pl = info_missing_players(collection, ST_INFO) #missing players from our db

In [13]:
#missing_pl = info_missing_players(collection, ST_INFO) #missing players from our d

In [14]:
database=ST_INFO
missing_ids = find_missing_players(collection, database)
all_pl = []
k=0
while k < len(missing_ids):
    idx = missing_ids[k]
    try:
        infos = personal_info(idx, database)
    except:
        k = k-1
        continue
    dic = {}
    dic['Id'] = idx
    dic['Classe'] = infos[0]
    dic['Eta\''] = infos[1]
    dic['Nome Completo'] = infos[2]
    dic['Nazionalita\''] = infos[3]
    all_pl.append(dic)
    print(k/len(missing_ids))
    k+=1
missing_pl=pd.DataFrame(all_pl, index = missing_ids)

0.0
0.030303030303030304
0.06060606060606061
0.09090909090909091
0.12121212121212122
0.15151515151515152
0.18181818181818182
0.21212121212121213
0.24242424242424243
0.2727272727272727
0.30303030303030304
0.3333333333333333
0.36363636363636365
0.3939393939393939
0.42424242424242425
0.45454545454545453
0.48484848484848486
0.5151515151515151
0.5454545454545454
0.5757575757575758
0.6060606060606061
0.6363636363636364
0.6666666666666666
0.696969696969697
0.7272727272727273
0.7575757575757576
0.7878787878787878
0.8181818181818182
0.8484848484848485
0.8787878787878788
0.9090909090909091
0.9393939393939394
0.9696969696969697


In [17]:
collection.find_one({'_id':4200})

{'_id': 4200,
 'name': 'GIROUD',
 'info': {'personal_info': {'full_name': 'Giroud Oliver',
   'birthdate': '30/09/1986',
   'birthdate_num': 19860930,
   'nation': 'Francia',
   'team_real': 'Milan',
   'FC_role': 'A'},
  'contract': {'owner': None,
   'start_date': '2021/07/30',
   'cost': 0,
   'acquisition_mode': None,
   'previous_owner': None,
   'quotation_initial': 20},
  'current_team': {'owner': None,
   'squad': None,
   'start_date': '2021/07/30',
   'previous_team': None,
   'quotation_initial': 20,
   'on_loan': False,
   'loan_info': None},
  'stats': {'Pg': 0,
   'Mv': 0,
   'Mf': 0,
   'Gf': 0,
   'Gs': 0,
   'Rp': 0,
   'Rc': 0,
   'R+': 0,
   'R-': 0,
   'Ass': 0,
   'Amm': 0,
   'Esp': 0,
   'Au': 0,
   'Qt_A': 20,
   'Qt_I': 20}}}

In [19]:
missing_pl

,Id,Classe,Eta',Nome Completo,Nazionalita'
481,481,19/08/1991,30,Brignoli Alberto,Italia
5448,5448,22/02/1993,28,Furlan Jacopo,Italia
4441,4441,02/01/1995,26,Andrenacci Lorenzo,Italia
2091,2091,12/03/2000,21,Plizzari Alessandro,Italia
2134,2134,12/04/1995,26,Falcone Wladimiro,Italia
316,316,10/03/1989,32,Berisha Etrit,"Albania, Kosovo"
4525,4525,03/07/2000,21,Gemello Luca,Italia
5478,5478,23/01/1985,36,Maenpaa Niki,Finlandia
5445,5445,02/09/2001,20,Binks Luis,"Inghilterra, Scozia"
5461,5461,16/07/2000,21,Kamenovic Dimitrije,Serbia


In [20]:
ST_INFO.rename(columns = {'Qt. A': 'Qt_A','Qt. I': 'Qt_I'}, inplace=True)
ST_INFO.to_pickle("Statistiche_giocatori.pkl")
missing_pl.to_pickle('Missing_Players.pkl')
ST_INFO

,Id,R,Nome,Squadra,Pg,Mv,Mf,Gf,Gs,Rp,...,R-,Ass,Amm,Esp,Au,Qt_A,Qt_I,Diff.,Nome Squadra,Allenatore
2531,2531,A,LUKAKU,Inter,0,0,0,0,0,0,...,0,0,0,0,0,44,44,0,SOROS FC,musci8
2610,2610,A,RONALDO,Juventus,0,0,0,0,0,0,...,0,0,0,0,0,44,44,0,AS 800A,enzo
507,507,A,MURIEL,Atalanta,0,0,0,0,0,0,...,0,0,0,0,0,39,39,0,PDG 1908,pietro
785,785,A,IMMOBILE,Lazio,0,0,0,0,0,0,...,0,0,0,0,0,38,38,0,PALLA PAZZA,nanni
2841,2841,A,VLAHOVIC,Fiorentina,0,0,0,0,0,0,...,0,0,0,0,0,34,34,0,SOROS FC,musci8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4936,4936,A,MANZARI,Sassuolo,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,svincolato,None
5378,5378,A,ODDEI,Sassuolo,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,svincolato,None
5477,5477,A,WARMING,Torino,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,svincolato,None
5493,5493,A,BOCALON,Venezia,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,svincolato,None


In [21]:
df=pd.read_pickle('Statistiche_giocatori.pkl')

In [26]:
df.iloc[180]

Id                     644
R                        C
Nome                PULGAR
Squadra         Fiorentina
Pg                       0
Mv                       0
Mf                       0
Gf                       0
Gs                       0
Rp                       0
Rc                       0
R+                       0
R-                       0
Ass                      0
Amm                      0
Esp                      0
Au                       0
Qt_A                     8
Qt_I                     8
Diff.                    0
Nome Squadra    svincolato
Allenatore            None
Name: 644, dtype: object

In [21]:
16*2.5

40.0